# Imports

In [1]:
from ROOT import TFile, TTree
import numpy as np
import pandas as pd

%jsroot on

Welcome to JupyROOT 6.24/06


# Load data

In [5]:
fin = TFile("HiForestAOD.root")
t = fin.Get("ak4PFJetAnalyzer/t")
t.AddFriend("hiEvtAnalyzer/HiTree") # to get the evt weight

In [6]:
t.Show(0)

======> EVENT:0
 nref            = 4
 rawpt           = 83.0575, 
                  32.4569, 24.1664, 24.0002
 jtpt            = 88.9238, 
                  36.8545, 28.2229, 26.4539
 jteta           = -0.533781, 
                  -1.13663, -1.69058, -0.141773
 jtphi           = -1.1192, 
                  2.19589, 1.99657, -0.257372
 jtHadFlav       = 0, 
                  0, 0, 5
 jtParFlav       = 21, 
                  21, 21, -5
 jtNbHad         = 0, 
                  0, 0, 1
 jtNcHad         = 0, 
                  0, 0, 1
 jtNbPar         = 0, 
                  0, 0, 1
 jtNcPar         = 0, 
                  0, 0, 0
 jtHasGSPB       = 0, 
                  0, 0, 1
 jtHasGSPC       = 0, 
                  0, 0, 0
 jtIsHardest     = 1, 
                  0, 0, 1
 sjt1E           = 48.9985, 
                  25.3854, 48.0666, 4.98427
 sjt1Pt          = 41.9645, 
                  15.8083, 16.3513, 4.81789
 sjt1Eta         = -0.567548, 
                  -1.04605, -1.73963, -0.

# Create histograms

In [3]:
from ROOT import TH1D, TCanvas

Count the percentage of tracks coming from B decay products that are associated to a secondary vertex (SV).

In [ ]:
hsv = TH1D("hsv", "percentage of tracks from B products associated to any SV per event", 25, 0, 100)
hsv.SetXTitle("% b-product track in SVs per event")
hsv.SetXTitle("nb of events")

nentries = t.GetEntries()

for ient in range(nentries):
    ient = int(ient)
    
    t.GetEntry(ient)
    
    bProducts = 0
    bProductsInSV = 0
    for itrack in range(t.nIP):
        itrack = int(itrack)
        
        # Keep only B decay products
        sta = t.ipMatchStatus[itrack]
        if (sta < 100): 
            continue
        bProducts += 1
        
        # Look for track in SV
        eps = 1e-3
        
        trackFoundInSV = False
        for svEta, svPhi in zip(np.array(t.svtxTrEta).flatten(), np.array(t.svtxTrPhi).flatten()):
            if ((t.ipEta[itrack] - svEta) > eps): 
                continue
            if ((t.ipPhi[itrack] - svPhi) > eps): 
                continue
            trackFoundInSV = True
            break
            
        if trackFoundInSV:
            bProductsInSV += 1
    hsv.Fill(bProductsInSV / bProducts, t.weight)       
    
c = TCanvas("c", "c", 800, 800)
hsv.Draw("hist")
c.Draw()